In [1]:
# ENTER DATE RANGE
#
# note: enddate should be +1 with the actual date.
# example: daterange is Sept 1-15
#          startdate = 1
#          enddate = 16


In [2]:
startdate = 14
enddate = 15

In [4]:
import jira_worklogs_lib as jira
import json
import pandas as pd
import math
import numpy as np
from time import sleep

In [5]:
# use to load jira credential for security reasons

path_to_json = "credential_template.json"
with open(path_to_json, "r") as handler:
    info = json.load(handler)
jira_aut = jira.jira_worklog()
jira_aut.login(info['users'], info['passwords'])

In [6]:
#preparing the worklog template file

df = pd.read_excel('Worklog Entry Template.xlsx', sheet_name='worklog Template')
df.columns = [str(x).strip() for x in df.loc[4,:].values]
df = df.loc[5:,:]
df.reset_index(drop= True, inplace= True)
df.dropna(subset=['Activity Code'], inplace = True)
df = df.loc[4:,:'Comments']
df['hour'] = df.apply(lambda x: math.floor(x['Time spent (Hr)']), axis = 1)
df['minutes'] = df.apply(lambda x: round((x['Time spent (Hr)'] - math.floor(x['Time spent (Hr)'])) * 60), axis = 1)
df['Jira Time spent'] = df.apply(lambda x: str(x['hour']) + 'h '+ str(x['minutes']) + 'm', axis= 1)
df['Description'] = df.apply(lambda x: str(x['Activity Code']) + ' | '+ str(x['Comments']), axis= 1)
df['Date'] = pd.to_datetime(df['Date'])
df['Jira Date'] = df.apply(lambda x: x['Date'].strftime("%m/%d/%y"), axis= 1)

In [8]:
# merging the links from the reference file to the main dataframe

jira_ref = pd.read_csv('Jira Links.csv')
df = df.merge(jira_ref)

In [11]:
df['Link'] = np.where(df['Project description'].isnull(), 
                                       df['Link'], 
                                       "https://legalmatch.atlassian.net/browse/" + df['Project description"])

In [13]:
df

,Date,Activity Code,Business area,Project description,Task,Time spent (Hr),Comments,hour,minutes,Jira Time spent,Description,Jira Date,Link
0,2022-09-09,SO-REP,Sales/Membership,Call Monitoring,Task implementation,1.5,Call Monitoring Stats,1,30,1h 30m,SO-REP | Call Monitoring Stats,09/09/22,https://legalmatch.atlassian.net/browse/Call M...
1,2022-09-09,SO-REP,Sales/Membership,Call Monitoring,Task implementation,1,Call Monitoring Assignments,1,0,1h 0m,SO-REP | Call Monitoring Assignments,09/09/22,https://legalmatch.atlassian.net/browse/Call M...
2,2022-09-12,SO-REP,Sales/Membership,Call Monitoring,Task implementation,1.5,Call Monitoring Stats,1,30,1h 30m,SO-REP | Call Monitoring Stats,09/12/22,https://legalmatch.atlassian.net/browse/Call M...
3,2022-09-12,SO-REP,Sales/Membership,Call Monitoring,Task implementation,1,Call Monitoring Assignments,1,0,1h 0m,SO-REP | Call Monitoring Assignments,09/12/22,https://legalmatch.atlassian.net/browse/Call M...
4,2022-09-13,SO-REP,Sales/Membership,Call Monitoring,Task implementation,1.5,Call Monitoring Stats,1,30,1h 30m,SO-REP | Call Monitoring Stats,09/13/22,https://legalmatch.atlassian.net/browse/Call M...
5,2022-09-13,SO-REP,Sales/Membership,Call Monitoring,Task implementation,1,Call Monitoring Assignments,1,0,1h 0m,SO-REP | Call Monitoring Assignments,09/13/22,https://legalmatch.atlassian.net/browse/Call M...
6,2022-09-14,SO-REP,Sales/Membership,Call Monitoring,Task implementation,1.5,Call Monitoring Stats,1,30,1h 30m,SO-REP | Call Monitoring Stats,09/14/22,https://legalmatch.atlassian.net/browse/Call M...
7,2022-09-14,SO-REP,Sales/Membership,Call Monitoring,Task implementation,1,Call Monitoring Assignments,1,0,1h 0m,SO-REP | Call Monitoring Assignments,09/14/22,https://legalmatch.atlassian.net/browse/Call M...
8,2022-09-09,SO-SF,Sales/Membership,NaN,Task implementation,5,Avvo scraping,5,0,5h 0m,SO-SF | Avvo scraping,09/09/22,https://legalmatch.atlassian.net/browse/SAL-240
9,2022-09-09,SO-SF,Sales/Membership,NaN,Task implementation,0.5,DemandTools Match Module Process,0,30,0h 30m,SO-SF | DemandTools Match Module Process,09/09/22,https://legalmatch.atlassian.net/browse/SAL-240


In [ ]:
# looping in the dataframe entry

sleep(5)
for i in range(startdate, enddate):
    print(i)
    day_str = str(i)
    for index, row in df[df['Date'] == pd.to_datetime('9/' + day_str + '/2022')].iterrows():
        print(row['Link'], row['Jira Date'],row['Jira Time spent'], row['Description'])
        jira_aut.log_time(row['Link'], row['Jira Date'],row['Jira Time spent'], row['Description'])
        sleep(5)